In [14]:
import pyodbc
import pandas as pd
import numpy as np
import os

# Configuración de conexiones
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'
AZURE_AUTH = 'Authentication=ActiveDirectoryInteractive'

LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

# Conexión a Azure SQL con opción alternativa
def get_azure_connection():
    try:
        print("🔄 Intentando conectar a Azure SQL...")
        conn = pyodbc.connect(f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};{AZURE_AUTH}", timeout=10)
        print("✅ Conexión a Azure SQL establecida.")
        return conn
    except Exception as e:
        print(f"❌ No se pudo conectar a Azure SQL. Error: {e}")
        return None

# Conexión a SQL Server Local
def get_local_connection():
    try:
        print("🔄 Intentando conectar a SQL Server Local...")
        conn = pyodbc.connect(f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes", timeout=10)
        print("✅ Conexión a SQL Server Local establecida.")
        return conn
    except Exception as e:
        print(f"❌ No se pudo conectar a SQL Server Local. Error: {e}")
        return None

# Función para leer consultas SQL desde archivo
def leer_consulta_desde_archivo(archivo):
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"❌ Error al leer el archivo SQL {archivo}: {e}")
        return None

# Procesar una tabla
def procesar_tabla(nombre_tabla, archivo_sql):
    print(f"\n📌 Procesando tabla: {nombre_tabla}")
    
    sql_query = leer_consulta_desde_archivo(archivo_sql)
    if not sql_query:
        return False

    conn_azure = get_azure_connection()
    if not conn_azure:
        return False

    try:
        df = pd.read_sql(sql_query, conn_azure)
        if df.empty:
            print(f"⚠️ La consulta {nombre_tabla} no devolvió datos.")
            return False

        df.fillna(0, inplace=True)
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)

        conn_local = get_local_connection()
        if not conn_local:
            return False

        with conn_local.cursor() as cursor:
            cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla}")
            conn_local.commit()
            print(f"🗑️ Tabla {nombre_tabla} eliminada (si existía).")

            create_table_sql = f"""
            CREATE TABLE {nombre_tabla} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f"✅ Tabla {nombre_tabla} creada correctamente.")

            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {nombre_tabla} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f"✅ {df.shape[0]} filas insertadas en {nombre_tabla}.")

        return True

    except Exception as e:
        print(f"❌ Error procesando {nombre_tabla}: {e}")
        return False

# Lista de tablas a importar
TABLAS_A_IMPORTAR = {
    "DATAEX.FACT_SALES": "../consultas_dimensional/DATAEX.FACT_SALES.sql",
    "DATAEX.DIM_CLIENTE": "../consultas_dimensional/DATAEX.DIM_CLIENTE.sql",
    "DATAEX.DIM_PRODUCTO": "../consultas_dimensional/DATAEX.DIM_PRODUCTO.sql",
    "DATAEX.DIM_LUGAR": "../consultas_dimensional/DATAEX.DIM_LUGAR.sql",
    "DATAEX.DIM_TIEMPO": "../consultas_dimensional/DATAEX.DIM_TIEMPO.sql",
}

# Ejecutar el script
if __name__ == "__main__":
    print("\n🚀 INICIANDO IMPORTACIÓN DE TABLAS")

    exitosas = 0
    for tabla, sql_file in TABLAS_A_IMPORTAR.items():
        if procesar_tabla(tabla, sql_file):
            exitosas += 1

    print("\n✅ RESUMEN DE EJECUCIÓN")
    print(f"✔️ Tablas procesadas: {len(TABLAS_A_IMPORTAR)}")
    print(f"✔️ Importaciones exitosas: {exitosas}")
    print(f"❌ Errores: {len(TABLAS_A_IMPORTAR) - exitosas}")
    print("🎯 Proceso finalizado.")



🚀 INICIANDO IMPORTACIÓN DE TABLAS

📌 Procesando tabla: DATAEX.FACT_SALES
🔄 Intentando conectar a Azure SQL...
✅ Conexión a Azure SQL establecida.


C:\Users\josit\AppData\Local\Temp\ipykernel_29724\3968752722.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


🔄 Intentando conectar a SQL Server Local...
✅ Conexión a SQL Server Local establecida.
🗑️ Tabla DATAEX.FACT_SALES eliminada (si existía).
✅ Tabla DATAEX.FACT_SALES creada correctamente.
✅ 58049 filas insertadas en DATAEX.FACT_SALES.

📌 Procesando tabla: DATAEX.DIM_CLIENTE
🔄 Intentando conectar a Azure SQL...
✅ Conexión a Azure SQL establecida.


C:\Users\josit\AppData\Local\Temp\ipykernel_29724\3968752722.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


🔄 Intentando conectar a SQL Server Local...
✅ Conexión a SQL Server Local establecida.
🗑️ Tabla DATAEX.DIM_CLIENTE eliminada (si existía).
✅ Tabla DATAEX.DIM_CLIENTE creada correctamente.
✅ 44053 filas insertadas en DATAEX.DIM_CLIENTE.

📌 Procesando tabla: DATAEX.DIM_PRODUCTO
🔄 Intentando conectar a Azure SQL...
✅ Conexión a Azure SQL establecida.
🔄 Intentando conectar a SQL Server Local...
✅ Conexión a SQL Server Local establecida.
🗑️ Tabla DATAEX.DIM_PRODUCTO eliminada (si existía).
✅ Tabla DATAEX.DIM_PRODUCTO creada correctamente.


...

✅ 404 filas insertadas en DATAEX.DIM_PRODUCTO.

📌 Procesando tabla: DATAEX.DIM_LUGAR
🔄 Intentando conectar a Azure SQL...
✅ Conexión a Azure SQL establecida.
🔄 Intentando conectar a SQL Server Local...
✅ Conexión a SQL Server Local establecida.
🗑️ Tabla DATAEX.DIM_LUGAR eliminada (si existía).
✅ Tabla DATAEX.DIM_LUGAR creada correctamente.
✅ 12 filas insertadas en DATAEX.DIM_LUGAR.


C:\Users\josit\AppData\Local\Temp\ipykernel_29724\3968752722.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)



📌 Procesando tabla: DATAEX.DIM_TIEMPO
🔄 Intentando conectar a Azure SQL...
✅ Conexión a Azure SQL establecida.


C:\Users\josit\AppData\Local\Temp\ipykernel_29724\3968752722.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn_azure)


🔄 Intentando conectar a SQL Server Local...
✅ Conexión a SQL Server Local establecida.
🗑️ Tabla DATAEX.DIM_TIEMPO eliminada (si existía).
✅ Tabla DATAEX.DIM_TIEMPO creada correctamente.
✅ 3652 filas insertadas en DATAEX.DIM_TIEMPO.

✅ RESUMEN DE EJECUCIÓN
✔️ Tablas procesadas: 5
✔️ Importaciones exitosas: 5
❌ Errores: 0
🎯 Proceso finalizado.


In [15]:
import pyodbc
import os
import re

def ejecutar_script_sql_en_sqlserver():
    try:
        # Configuración de conexión
        SERVER = 'localhost'
        DATABASE = 'dwh_case1'
        DRIVER = '{ODBC Driver 17 for SQL Server}'
        conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
        
        # Archivo SQL a procesar
        archivo_sql = '../claves/FACT/VALORES_NULOS_FACTS.sql'
        
        if not os.path.exists(archivo_sql):
            raise FileNotFoundError(f"El archivo {archivo_sql} no existe")

        print("Conectando a SQL Server...")
        with pyodbc.connect(conn_str) as conn:
            conn.autocommit = False
            cursor = conn.cursor()
            
            # Leer y procesar el archivo SQL correctamente
            with open(archivo_sql, 'r', encoding='utf-8') as file:
                contenido = file.read()
                
                # 1. Eliminar comentarios (-- y /* */)
                contenido = re.sub(r'--.*?\n', '', contenido)  # Comentarios de línea
                contenido = re.sub(r'/\*.*?\*/', '', contenido, flags=re.DOTALL)  # Bloques de comentarios
                
                # 2. Separar sentencias individuales correctamente
                sentencias = []
                buffer = ""
                for linea in contenido.split('\n'):
                    linea = linea.strip()
                    if not linea:
                        continue
                    
                    buffer += " " + linea if buffer else linea
                    
                    # Verificar si tenemos una sentencia completa
                    if ';' in buffer:
                        partes = buffer.split(';')
                        for parte in partes[:-1]:
                            parte = parte.strip()
                            if parte:
                                sentencias.append(parte)
                        buffer = partes[-1].strip()
                
                # Añadir cualquier sentencia restante
                if buffer.strip():
                    sentencias.append(buffer.strip())
            
            # Filtrar y ejecutar solo sentencias ALTER COLUMN válidas
            for i, sentencia in enumerate(sentencias, 1):
                try:
                    # Validar que sea una sentencia ALTER TABLE ALTER COLUMN
                    if not re.match(r'^\s*ALTER\s+TABLE\s+\w+\.\w+\s+ALTER\s+COLUMN', sentencia, re.IGNORECASE):
                        print(f"\n[{i}] Omitiendo (no es ALTER COLUMN válido): {sentencia[:100]}...")
                        continue
                        
                    print(f"\n[{i}] Ejecutando: {sentencia[:100]}...")
                    
                    cursor.execute(sentencia)
                    conn.commit()
                    print("  ✓ Éxito")
                    
                except pyodbc.Error as e:
                    print(f"\n❌ Error en la sentencia {i}:")
                    print(f"Código SQL: {e.args[0]}")
                    print(f"Mensaje: {e.args[1]}")
                    print(f"Sentencia completa:\n{sentencia}")
                    
                    # Manejo especial para errores de conversión de NULL a NOT NULL
                    if "cannot alter column" in str(e.args[1]).lower() and "because one or more objects access this column" in str(e.args[1]).lower():
                        print("\n⚠️ Posible solución: Eliminar constraints dependientes antes de modificar la columna")
                    
                    continuar = input("\n¿Continuar con las siguientes sentencias? (s/n): ").lower()
                    if continuar != 's':
                        conn.rollback()
                        print("Proceso cancelado por el usuario")
                        return
            
            print("\n¡Proceso completado con éxito!")
            
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()
    finally:
        if 'conn' in locals():
            conn.close()

# Ejecutar la función
ejecutar_script_sql_en_sqlserver()

Conectando a SQL Server...

[1] Ejecutando: ALTER TABLE DATAEX.DIM_LUGAR ALTER COLUMN TIENDA_ID int NOT NULL...
  ✓ Éxito

[2] Ejecutando: ALTER TABLE DATAEX.DIM_PRODUCTO ALTER COLUMN Id_Producto nvarchar(255) NOT NULL...
  ✓ Éxito

[3] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN CODE nvarchar(255) NOT NULL...
  ✓ Éxito

[4] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN Sales_Date nvarchar(255) NOT NULL...
  ✓ Éxito

[5] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN Customer_ID int NOT NULL...
  ✓ Éxito

[6] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN Id_Producto nvarchar(255) NOT NULL...
  ✓ Éxito

[7] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN PVP int NOT NULL...
  ✓ Éxito

[8] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN IMPUESTOS int NOT NULL...
  ✓ Éxito

[9] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COLUMN COSTE_VENTA_NO_IMPUESTOS int NOT NULL...
  ✓ Éxito

[10] Ejecutando: ALTER TABLE DATAEX.FACT_SALES ALTER COL

In [16]:
import pyodbc

# Conexión a SQL Server LOCAL
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'  
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Ruta del archivo SQL
SQL_FILE = "../claves/FACT/INT.sql"

def leer_sql(archivo):
    """Lee la consulta SQL desde el archivo"""
    with open(archivo, 'r', encoding='utf-8') as file:
        return file.read()

try:
    # Leer el archivo SQL
    print(f"📂 Leyendo consulta en {SQL_FILE}...")
    sql_query = leer_sql(SQL_FILE)
    
    # Conectar a SQL Server Local
    print(f"🔗 Conectando a SQL Server Local ({LOCAL_DATABASE})...")
    conn_local = pyodbc.connect(local_conn_str)
    cursor = conn_local.cursor()
    
    # Ejecutar la consulta
    print(f"⚡ Ejecutando consulta...")
    cursor.execute(sql_query)
    conn_local.commit()

    print("✅ Consulta ejecutada correctamente en dwh_case1!")

except Exception as e:
    print(f"❌ Error: {e}")
    if 'conn_local' in locals():
        conn_local.rollback()

finally:
    if 'conn_local' in locals():
        conn_local.close()


📂 Leyendo consulta en ../claves/FACT/INT.sql...
🔗 Conectando a SQL Server Local (dwh_case1)...
⚡ Ejecutando consulta...
✅ Consulta ejecutada correctamente en dwh_case1!


In [17]:
import pyodbc
import os

def aplicar_constraints():
    # Configuración de conexión
    SERVER = 'localhost'
    DATABASE = 'dwh_case1'
    DRIVER = '{ODBC Driver 17 for SQL Server}'
    conn_str = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes'
    
    # Nombre del archivo SQL
    SQL_FILE = "../claves/FACT/PKS_FACT_SALES.sql"
    
    try:
        # Verificar que el archivo existe
        if not os.path.exists(SQL_FILE):
            raise FileNotFoundError(f"El archivo {SQL_FILE} no existe")
        
        # Leer el contenido del archivo SQL
        with open(SQL_FILE, 'r', encoding='utf-8') as file:
            sql_script = file.read()
        
        # Conectar a SQL Server
        print("Conectando a SQL Server...")
        with pyodbc.connect(conn_str) as conn:
            conn.autocommit = False  # Importante para el manejo de transacciones
            cursor = conn.cursor()
            
            # Verificar si la tabla existe
            cursor.execute("""
                SELECT COUNT(*) 
                FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_SCHEMA = 'DATAEX' 
                AND TABLE_NAME = 'FACT_SALES'
            """)
            if cursor.fetchone()[0] == 0:
                raise Exception("La tabla DATAEX.FACT_SALES no existe")
            
            # Separar las sentencias SQL
            sentencias = [s.strip() for s in sql_script.split(';') if s.strip()]
            
            for i, sentencia in enumerate(sentencias, 1):
                try:
                    if not sentencia:
                        continue
                        
                    # Mostrar la sentencia (sin saltos de línea para evitar el error)
                    print(f"\n[{i}] Ejecutando: {sentencia[:50].replace(chr(10), ' ')}...")
                    
                    # Verificación especial para FOREIGN KEYS
                    if "FOREIGN KEY" in sentencia.upper():
                        tabla_ref = sentencia.upper().split("REFERENCES")[1].split("(")[0].strip()
                        print(f"  → Verificando integridad referencial con {tabla_ref}...")
                        
                        # Ejecutar con WITH CHECK para forzar validación
                        sentencia = sentencia.replace(");", ") WITH CHECK;")
                    
                    cursor.execute(sentencia)
                    conn.commit()
                    print("  ✓ Éxito")
                    
                except pyodbc.Error as e:
                    print(f"\n❌ Error en la sentencia {i}:")
                    print(f"Código SQL: {e.args[0]}")
                    print(f"Mensaje: {e.args[1]}")
                    
                    # Opción para intentar con NOCHECK si falla
                    if "FOREIGN KEY" in sentencia.upper():
                        print("\nIntentando con WITH NOCHECK...")
                        try:
                            sentencia_nocheck = sentencia.replace(");", ") WITH NOCHECK;")
                            cursor.execute(sentencia_nocheck)
                            conn.commit()
                            print("  ✓ Aplicada con NOCHECK (no se validaron datos existentes)")
                        except pyodbc.Error as e2:
                            print(f"  ✖ Error con NOCHECK: {e2.args[1]}")
                            conn.rollback()
                    
                    continuar = input("\n¿Continuar con las siguientes constraints? (s/n): ").lower()
                    if continuar != 's':
                        conn.rollback()
                        print("Proceso cancelado por el usuario")
                        return
            
            # Verificación final de constraints aplicadas
            print("\nResumen de constraints aplicadas:")
            cursor.execute("""
                SELECT 
                    CONSTRAINT_NAME, 
                    CONSTRAINT_TYPE 
                FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS 
                WHERE TABLE_SCHEMA = 'DATAEX' 
                AND TABLE_NAME = 'FACT_SALES'
            """)
            for row in cursor:
                print(f"  - {row.CONSTRAINT_NAME}: {row.CONSTRAINT_TYPE}")
            
            print("\n¡Proceso completado!")
    
    except Exception as e:
        print(f"\n⚠️ Error general: {str(e)}")
        if 'conn' in locals():
            conn.rollback()

# Ejecutar la función
aplicar_constraints()

Conectando a SQL Server...

[1] Ejecutando: ---Ponemos para que date sea la pk de la tabla dim...
  ✓ Éxito

[2] Ejecutando: ---Ponemos para que date sea la pk de la tabla dim...
  ✓ Éxito

[3] Ejecutando: ---Ponemos para que date sea la pk de la tabla dim...
  ✓ Éxito

[4] Ejecutando: -- Establecer Customer_ID como clave foránea que r...
  ✓ Éxito

[5] Ejecutando: -- Establecer Customer_ID como clave foránea que r...
  → Verificando integridad referencial con DATAEX.DIM_CLIENTE...
  ✓ Éxito

[6] Ejecutando: -- Establecer Id_Producto como clave foránea que r...
  → Verificando integridad referencial con DATAEX.DIM_PRODUCTO...
  ✓ Éxito

[7] Ejecutando: -- Establecer Sales_Date como clave foránea que re...
  → Verificando integridad referencial con DATAEX.DIM_TIEMPO...
  ✓ Éxito

[8] Ejecutando: -- Establecer TIENDA_ID como clave foránea que ref...
  → Verificando integridad referencial con DATAEX.DIM_LUGAR...
  ✓ Éxito

Resumen de constraints aplicadas:
  - PK_DATAEXFACT_SALES: PRIMARY

In [18]:
import pyodbc

def crear_columna_churn():
    try:
        # Configuración de conexión
        conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=dwh_case1;Trusted_Connection=yes;TrustServerCertificate=yes'
        
        with pyodbc.connect(conn_str) as conn:
            cursor = conn.cursor()
            
            # Verificar y añadir columna si no existe
            cursor.execute("""
                IF NOT EXISTS (
                    SELECT * 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_SCHEMA = 'DATAEX' 
                    AND TABLE_NAME = 'FACT_SALES' 
                    AND COLUMN_NAME = 'churn'
                )
                BEGIN
                    ALTER TABLE DATAEX.FACT_SALES ADD churn INT NULL;
                    SELECT 'Columna churn añadida' AS Resultado;
                END
                ELSE
                BEGIN
                    SELECT 'La columna churn ya existe' AS Resultado;
                END
            """)
            print(cursor.fetchone()[0])
            
            # Actualizar valores de churn
            cursor.execute("""
                UPDATE DATAEX.FACT_SALES 
                SET churn = CASE 
                    WHEN KM_Ultima_Revision_Final IS NULL THEN 0
                    WHEN KM_Ultima_Revision_Final > 400 THEN 0
                    ELSE 1
                END
            """)
            conn.commit()
            print(f"Filas actualizadas: {cursor.rowcount}")
            
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        if 'conn' in locals():
            conn.close()

# Ejecutar la función
crear_columna_churn()

Columna churn añadida
Filas actualizadas: 58049
